In [2]:
## DATA MANIPULATION
import numpy as np
import pickle

## TF-IDF VECTORIZER
from sklearn.feature_extraction.text import TfidfVectorizer

## CLASSIFICATION
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

### LOAD PREPROCESSED DATA

In [3]:
f = open('../data/preprocessed.pkl','rb')
train, valid = pickle.load(f)

### CREATE BAG OF WORDS (TF-IDF) TRANSFORMER

In [7]:
tfidf = TfidfVectorizer(ngram_range=(1,2),
                        min_df=3, 
                        max_df=0.9, 
                        strip_accents='unicode', 
                        use_idf=1,
                        smooth_idf=1, 
                        sublinear_tf=1)

X_train = tfidf.fit_transform(train['comment_text'])      # vectorized train x
X_valid = tfidf.transform(valid['comment_text'])          # vectorized valid x

### STRONG BASELINE (LINEAR MODEL): NB-SVM
- based on Wang & Manning, 2012 (https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf)
- A SVM model (liblinear) that takes transformed features using Naive Bayes' log-count ratio

In [9]:
## NB-SVM MODEL
def NB_SVM(x,y):
    y = y.values
    sum_1 = x[y==1].sum(axis=0)+1           # Feature Sum for Class 1
    p_1 = (sum_1) / ((y==1).sum())          # Convert to ratio of feature in class 1 - p(f|1)

    sum_0 = x[y==0].sum(axis=0)+1           # Feature Sum for Class 0
    p_0 = (sum_0) / ((y==0).sum())          # Convert to ratio of feature in class 0 - p(f|0) 

    r = np.log(p_1/p_0)                     # Compute log ratios (the transformation matrix)
    x_nb = x.multiply(r)                    # Obtain NB feature
    
    m = LinearSVC(class_weight='balanced')  # Balanced class weights [n_samples / n_classes * np.bincount(y)]
    m.fit(x_nb,y)                           # Fit model
    return m , r                            # return fitted model & transformation matrix (need for X_valid / X_test)

labels = train.columns[2:]                  # Grab labels

## CREATE A MODEL FOR EACH CATEGORY
for label in labels:
    # Get model and transformation matrix for category
    m,r = NB_SVM(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid.multiply(r))
    # Evaluate predictions
    print('Validation accuracy for {0} comments is {1:.2f}, with precision score of {2:.2f} and recall score of {3:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds)))

Validation accuracy for toxic comments is 0.95, with precision score of 0.78 and recall score of 0.75
Validation accuracy for severe_toxic comments is 0.98, with precision score of 0.28 and recall score of 0.40
Validation accuracy for obscene comments is 0.98, with precision score of 0.78 and recall score of 0.76
Validation accuracy for threat comments is 1.00, with precision score of 0.52 and recall score of 0.34
Validation accuracy for insult comments is 0.97, with precision score of 0.67 and recall score of 0.66
Validation accuracy for identity_hate comments is 0.99, with precision score of 0.31 and recall score of 0.38


### LOGISTIC REGRESSION

In [10]:
for label in labels:
    # Create & Fit model
    m = LogisticRegression(solver='saga',class_weight='balanced')
    m.fit(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid)
    # Evaluate predictions
    print('Validation accuracy for {0} comments is {1:.2f}, with precision score of {2:.2f} and recall score of {3:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds)))

Validation accuracy for toxic comments is 0.93, with precision score of 0.62 and recall score of 0.86
Validation accuracy for severe_toxic comments is 0.93, with precision score of 0.12 and recall score of 0.93
Validation accuracy for obscene comments is 0.96, with precision score of 0.58 and recall score of 0.90
Validation accuracy for threat comments is 0.89, with precision score of 0.02 and recall score of 0.93
Validation accuracy for insult comments is 0.92, with precision score of 0.38 and recall score of 0.92
Validation accuracy for identity_hate comments is 0.87, with precision score of 0.06 and recall score of 0.94


### SVM

In [11]:
for label in labels:
    # Create & Fit model
    m = LinearSVC(class_weight='balanced')
    m.fit(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid)
    # Evaluate predictions
    print('Validation accuracy for {0} comments is {1:.2f}, with precision score of {2:.2f} and recall score of {3:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds)))

Validation accuracy for toxic comments is 0.95, with precision score of 0.75 and recall score of 0.77
Validation accuracy for severe_toxic comments is 0.98, with precision score of 0.33 and recall score of 0.56
Validation accuracy for obscene comments is 0.98, with precision score of 0.77 and recall score of 0.79
Validation accuracy for threat comments is 1.00, with precision score of 0.49 and recall score of 0.46
Validation accuracy for insult comments is 0.97, with precision score of 0.66 and recall score of 0.74
Validation accuracy for identity_hate comments is 0.99, with precision score of 0.39 and recall score of 0.50


### XGBoost

In [20]:
for label in labels:
    # Create & Fit model
    m = XGBClassifier(n_estimators=100,
                      scale_pos_weight= sum(train[label]==0) / sum(train[label]==1),
                      n_jobs=-1)
    m.fit(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid)
    # Evaluate predictions
    print('Validation accuracy for {0} comments is {1:.2f}, with precision score of {2:.2f} and recall score of {3:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds)))

Validation accuracy for toxic comments is 0.94, with precision score of 0.67 and recall score of 0.78
Validation accuracy for severe_toxic comments is 0.98, with precision score of 0.26 and recall score of 0.75
Validation accuracy for obscene comments is 0.97, with precision score of 0.70 and recall score of 0.85
Validation accuracy for threat comments is 0.99, with precision score of 0.22 and recall score of 0.59
Validation accuracy for insult comments is 0.96, with precision score of 0.57 and recall score of 0.81
Validation accuracy for identity_hate comments is 0.97, with precision score of 0.22 and recall score of 0.71
